In [1]:
from functools import reduce
import cv2
import os
from PIL import Image
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision



In [3]:
model_asset_path='/home/nick/model/selfie_multiclass_256x256.tflite'
img_folder = "/home/nick/webui/ComfyUI/output/94-body/"
images = os.listdir(img_folder)


BaseOptions = mp.tasks.BaseOptions
ImageSegmenter = mp.tasks.vision.ImageSegmenter
ImageSegmenterOptions = mp.tasks.vision.ImageSegmenterOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a image segmenter instance with the image mode:
for image in images:
    options = ImageSegmenterOptions(
        base_options=BaseOptions(model_asset_path=model_asset_path),
        running_mode=VisionRunningMode.IMAGE,
        output_category_mask=True)
    with ImageSegmenter.create_from_options(options) as segmenter:
        media_pipe_image = mp.Image.create_from_file(f"{img_folder}/{image}")
        segmented_masks = segmenter.segment(media_pipe_image)
        face_mask = segmented_masks.confidence_masks[3]
        
        image_data = media_pipe_image.numpy_view()
        image_shape = image_data.shape
    
        if image_shape[-1] == 3:
            image_shape = (image_shape[0], image_shape[1], 4)
    
        mask_background_array = np.zeros(image_shape, dtype=np.uint8)
        mask_background_array[:] = (255, 255, 255, 0)
    
        mask_foreground_array = np.zeros(image_shape, dtype=np.uint8)
        mask_foreground_array[:] = (0, 0, 0, 255)
    
        condition = np.stack((face_mask.numpy_view(),) * image_shape[-1], axis=-1) > 0.25
        mask_array = np.where(condition, mask_foreground_array, mask_background_array)
    
        mask_image = Image.fromarray(mask_array)
    
        mask_image.save(f"{img_folder}/{image}-face.png")
    
        acc_mask = segmented_masks.confidence_masks[5]
        
        image_data = media_pipe_image.numpy_view()
        image_shape = image_data.shape
    
        if image_shape[-1] == 3:
            image_shape = (image_shape[0], image_shape[1], 4)
    
        mask_background_array = np.zeros(image_shape, dtype=np.uint8)
        mask_background_array[:] = (255, 255, 255, 0)
    
        mask_foreground_array = np.zeros(image_shape, dtype=np.uint8)
        mask_foreground_array[:] = (0, 0, 0, 255)
    
        condition = np.stack((acc_mask.numpy_view(),) * image_shape[-1], axis=-1) > 0.25
        mask_array = np.where(condition, mask_foreground_array, mask_background_array)
    
        mask_image = Image.fromarray(mask_array)
    
        mask_image.save(f"{img_folder}/{image}-acc.png")


        body_mask = segmented_masks.confidence_masks[2]
        
        image_data = media_pipe_image.numpy_view()
        image_shape = image_data.shape
    
        if image_shape[-1] == 3:
            image_shape = (image_shape[0], image_shape[1], 4)
    
        mask_background_array = np.zeros(image_shape, dtype=np.uint8)
        mask_background_array[:] = (255, 255, 255, 0)
    
        mask_foreground_array = np.zeros(image_shape, dtype=np.uint8)
        mask_foreground_array[:] = (0, 0, 0, 255)
    
        condition = np.stack((body_mask.numpy_view(),) * image_shape[-1], axis=-1) > 0.25
        mask_array = np.where(condition, mask_foreground_array, mask_background_array)
    
        mask_image = Image.fromarray(mask_array)
    
        mask_image.save(f"{img_folder}/{image}-body.png")


I0000 00:00:1710728149.867828    4214 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1710728149.951978    4996 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 22.2.5), renderer: KABINI (, LLVM 15.0.6, DRM 2.50, 5.15.0-58-generic)
I0000 00:00:1710728161.957861    4214 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1710728161.976180    5002 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 22.2.5), renderer: KABINI (, LLVM 15.0.6, DRM 2.50, 5.15.0-58-generic)
I0000 00:00:1710728177.097097    4214 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1710728177.122682    5017 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 22.2.5), renderer: KABINI (, LLVM 15.0.6, DRM 2.50, 5.15.0-58-generic)
I0000 00:00:1710728186.783614    4214 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1710728186.789752    5023 gl_context.cc:344] GL version: 3.2

In [ ]:
BaseOptions = mp.tasks.BaseOptions
ImageSegmenter = mp.tasks.vision.ImageSegmenter
ImageSegmenterOptions = mp.tasks.vision.ImageSegmenterOptions
VisionRunningMode = mp.tasks.vision.RunningMode

bg = 'background'
hr = 'hair'
bd = 'body (skin)'
fc = 'face (skin)'
cl = 'clothes'
ot = 'accessories'

# Create a image segmenter instance with the image mode:
options = ImageSegmenterOptions(
    base_options=BaseOptions(model_asset_path='/home/nick/webui/sdw/models/mediapipe/selfie_multiclass_256x256.tflite'),
    running_mode=VisionRunningMode.IMAGE,
    output_category_mask=True)
with ImageSegmenter.create_from_options(options) as segmenter:

    # Retrieve the masks for the segmented image
    media_pipe_image = mp.Image.create_from_file('/home/nick/webui/comfy/output/ComfyUI_00052_.png')
    segmented_masks = segmenter.segment(media_pipe_image)

    mask_targets = ['bg']
    masks = []
    for i, target in mask_targets:


        if target == bg:
            mask_index = 0
        elif target == hr:
            mask_index = 1
        elif target == bd:
            mask_index = 2
        elif target == fc:
            mask_index = 3
        elif target == cl:
            mask_index = 4
        elif target == ot:
            mask_index = 5

        masks.append(segmented_masks.confidence_masks[mask_index])

    image_data = media_pipe_image.numpy_view()
    image_shape = image_data.shape

    # convert the image shape from "rgb" to "rgba" aka add the alpha channel
    if image_shape[-1] == 3:
        image_shape = (image_shape[0], image_shape[1], 4)

    mask_background_array = np.zeros(image_shape, dtype=np.uint8)
    mask_background_array[:] = (0, 0, 0, 255)

    mask_foreground_array = np.zeros(image_shape, dtype=np.uint8)
    mask_foreground_array[:] = (255, 255, 255, 255)

    mask_arrays = []
    for i, mask in enumerate(masks):
        condition = np.stack((mask.numpy_view(),) * image_shape[-1], axis=-1) > 0.25
        mask_array = np.where(condition, mask_foreground_array, mask_background_array)
        mask_arrays.append(mask_array)

    # Merge our masks taking the maximum from each
    merged_mask_arrays = reduce(np.maximum, mask_arrays)

    # Create the image
    mask_image = Image.fromarray(merged_mask_arrays)

    mask_image.save("face-glasses2.png")

